<a href="https://colab.research.google.com/github/spatiha/Eluvio-Challenge/blob/main/DS_asses.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import re
import pickle
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


In [ ]:
df = pd.read_csv('/content/Eluvio_DS_Challenge.csv')

In [ ]:
df.head()

,time_created,date_created,up_votes,down_votes,title,over_18,author,category
0,1201232046,2008-01-25,3,0,Scores killed in Pakistan clashes,False,polar,worldnews
1,1201232075,2008-01-25,2,0,Japan resumes refuelling mission,False,polar,worldnews
2,1201232523,2008-01-25,3,0,US presses Egypt on Gaza border,False,polar,worldnews
3,1201233290,2008-01-25,1,0,Jump-start economy: Give health care to all,False,fadi420,worldnews
4,1201274720,2008-01-25,4,0,Council of Europe bashes EU&UN terror blacklist,False,mhermans,worldnews


In [ ]:
len(df)

509236

In [ ]:
print(sum(df['category'] == "worldnews"))

509236


In [ ]:
print(sum(df["down_votes"] == 0))

509236


In [ ]:
df = df.drop("category", axis = 1)

In [ ]:
df = df.drop("down_votes", axis = 1)

In [ ]:
df = df.drop("time_created", axis = 1)

In [ ]:
df = df.drop("date_created", axis = 1)

In [ ]:
df.head()

,up_votes,title,over_18,author
0,3,Scores killed in Pakistan clashes,False,polar
1,2,Japan resumes refuelling mission,False,polar
2,3,US presses Egypt on Gaza border,False,polar
3,1,Jump-start economy: Give health care to all,False,fadi420
4,4,Council of Europe bashes EU&UN terror blacklist,False,mhermans


In [ ]:
len(set(df['author']))

85838

In [ ]:
import nltk

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from nltk.stem.snowball import SnowballStemmer

In [ ]:
stemmer = SnowballStemmer("english")

In [ ]:
def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems

In [ ]:
def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [ ]:
title = df.title.str.lower()

In [ ]:
def tokenized_stemmed(title):
    totalvocab_stemmed = []
    totalvocab_tokenized = []
    for i in title:
        allwords_stemmed = tokenize_and_stem(i) 
        totalvocab_stemmed.extend(allwords_stemmed) 

        allwords_tokenized = tokenize_only(i)
        totalvocab_tokenized.extend(allwords_tokenized)
    return totalvocab_stemmed, totalvocab_tokenized

In [ ]:
totalvocab_stemmed_, totalvocab_tokenized_ = tokenized_stemmed(title)

In [ ]:
print(len(totalvocab_stemmed_))

7194561


In [ ]:
import sklearn.feature_extraction.text as text
stopwords = nltk.corpus.stopwords.words('english')
my_stop_words = text.ENGLISH_STOP_WORDS.union(stopwords)

In [ ]:
# tf-idf vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfidf_vectorizer = TfidfVectorizer(min_df =10**-3 ,analyzer = 'word', max_features=len(set(totalvocab_stemmed_)), stop_words=my_stop_words, tokenizer=tokenize_and_stem, ngram_range=(1,3))

In [ ]:
tfidf_matrix = tfidf_vectorizer.fit_transform(title)

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'s", 'abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'doe', 'dure', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'forti', 'henc', 'hereaft', 'herebi', 'howev', 'hundr', 'inde', 'mani', 'meanwhil', 'moreov', "n't", 'need', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'onli', 'otherwis', 'ourselv', 'perhap', 'pleas', 'sever', 'sha', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'themselv', 'thenc', 'thereaft', 'therebi', 'therefor', 'togeth', 'twelv', 'twenti', 'veri', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev', 'whi', 'wo', 'yourselv'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [ ]:
print(tfidf_matrix.shape)

(509236, 1814)


In [ ]:
#y = np.array(y)

In [ ]:
#y = [1 if i > thre else 0 for i in df['up_votes']]

In [ ]:
y = df['up_votes']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix, y, test_size = 0.2, shuffle = True, random_state = 42)

In [ ]:

#create naive baseline
from sklearn.metrics import mean_squared_error, r2_score
mean_price = np.mean(y)
y_test_naive = np.repeat(mean_price, len(y_test))
naive_rmse = mean_squared_error(y_test, y_test_naive)
naive_rmse = np.sqrt(naive_rmse)

print('Naive Baseline RMSE on test set: ' + str(round(naive_rmse,2)))


Naive Baseline RMSE on test set: 539.11


In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

steps = [('linear',LinearRegression())]
pipeline = Pipeline(steps)

# evaluate default model with 5 fold cross validation on train set
scores = cross_val_score(pipeline,X_train,y_train, cv=5, scoring='neg_root_mean_squared_error')
mean_score = abs(np.mean(scores))

#fit model on entire train set and predict on test set
pipeline.fit(X_train,y_train)
y_pred_def = pipeline.predict(X_test)

# evaluate predictions made on test set
def_rmse = mean_squared_error(y_test, y_pred_def)
def_rmse = np.sqrt(def_rmse)
def_r2 = r2_score(y_test,y_pred_def)

print('Linear Regression (default parameters) mean cross validated RMSE on train set: ' + str(round(mean_score,2)))
print()
print("Linear Regression (default parameters) RMSE on test set: " + str(round(def_rmse,2)))

Linear Regression (default parameters) mean cross validated RMSE on train set: 534.6

Linear Regression (default parameters) RMSE on test set: 531.41


In [ ]:
#evaluate svr with default parameters on test set
from sklearn.svm import SVR
from sklearn.preprocessing import RobustScaler,PowerTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

steps = [ ('SVR', SVR())]
pipeline = Pipeline(steps)

# evaluate default model with 5 fold cross validation on train set
scores = cross_val_score(pipeline,X_train,y_train, cv=5, scoring='neg_root_mean_squared_error')
mean_score = abs(np.mean(scores))

#fit model on entire train set and predict on test set
pipeline.fit(X_train,y_train)
y_pred_def = pipeline.predict(X_test)

# evaluate predictions made on test set
svr_def_r2 = r2_score(y_test,y_pred_def)
svr_def_rmse = np.sqrt(mean_squared_error(y_test, y_pred_def))

print('SVR (default parameters) mean cross validated RMSE on train set: ' + str(round(mean_score,2)))
print()
print('SVR (default parameters) R2 score on test set: ' + str(round(svr_def_r2,1)))
print()
print('SVR (default parameters) RMSE score on test set: ' + str(round(svr_def_rmse,2)))

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

# build random forest model
model = RandomForestRegressor(random_state=123)

# evaluate default model with 5 fold cross validation on train set
scores = cross_val_score(model,X_train,y_train, cv=5, scoring='neg_root_mean_squared_error')
mean_score = abs(np.mean(scores))

#fit model on entire train set and predict on test set
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# evaluate predictions made on test set
rf_def_rmse = np.sqrt(mean_squared_error(y_test, y_pred))
rf_def_r2 = r2_score(y_test,y_pred)

print('Random Forest (default parameters) mean cross validated RMSE on train set: ' + str(round(mean_score,2)))
print()
print('Random Forest (default parameters) R2 score on test set: ' + str(round(rf_def_r2,1)))
print()
print("Random Forest (default parameters) RMSE on test set: " + str(round(rf_def_rmse,2)))